## Mapping an explicit-hydrogen atomistic topology to a coarse-grained topology

In this example, we're going to learn how to use the TopologyMapper class to map an atomistic topology to a coarse-grained one.

In [1]:
import mdtraj as md

from cgtools.mapping import TopologyMapper

> Preparing to load module geomlib
> Found binary for source of module geomlib
> Finished loading module geomlib (took 0.001 s)
> Preparing to load module maplib
> Found binary for source of module maplib
> Finished loading module maplib (took 0.000 s)
> openmm or mdtraj did not load; these features will not work!


Initialize a `TopologyMapper` object using either an `mdtraj.Topology` object or a path to a pdb file. The topology in this pdb file has two water molecules, two dodecane molecules, and two poly butyl acrylate (PBA) 5-mers, 

In [2]:
# using pdb file
tm = TopologyMapper("solution.pdb")

# using mdtraj.Topology object
top = md.load("solution.pdb").topology
tm = TopologyMapper(top)

We will introduce a number of different ways to map residues. For the water residues, which are named `H2O` in the pdb file, we will perform the simplest mapping possible by mapping the residue to a single bead with the same name.

In [3]:
tm.add_residue_map('H2O', 'H2O')

For the dodecane molecules, which are named `DOD` in the pdb file, we demonstrate a number of ways to map each residue. For this first mapping, we can map the entire dodecane molecule to a single bead.

In [4]:
tm.add_residue_map('DOD', 'DOD')

For the next possible mapping dodecane, we use a two-bead mapping for each residue where each bead has the name `C6` and represents 6 heavy (non-H) atoms. The second argument is now a list of strings and we have to pass a list for the value of a an argument called `n_heavy_atoms_per_cg_bead`, which indicates the number of heavy atoms mapped to each CG bead.

In [5]:
tm.add_residue_map('DOD',
                   ['C6', 'C6'],
                   n_heavy_atoms_per_cg_bead=[6, 6],
                   replace=True)

Note that we had to pass `replace=True` keyword argument because a mapping for the `DOD` residue already existed in the `TopologyMapper` object.

For the final mapping we will consider for dodecane, we map dodecane to a 3-bead model.

In [6]:
tm.add_residue_map('DOD',
                   ['C4', 'C4', 'C4'],
                   n_heavy_atoms_per_cg_bead=[4, 4, 4],
                   replace=True)

We will stick with this mapping for the rest of this example.

For the poly butyl acrylate 5-mers, each chain has 5 residues named `A4`, each of which represent a monomer in the chain.
We map each monomer to two beads, where the acrylate group is represented by a bead named `Bpba` and the acrylate bead is represented by the same `C4` bead used in the dodecane.

In [7]:
tm.add_residue_map('A4',
                   ['Bpba', 'C4'],
                   n_heavy_atoms_per_cg_bead=[5, 4])

Once all of the mappings have been added, we can call the `create_map` method to create the mapping for the topology.

In [8]:
tm.create_map()

After the mapping is created, we can access information about the CG system with the attributes `cg_beads` and `cg_bonds`.
The `cg_beads` attributes returns an iterator that iterates through `CGBead` objects. For each `CGBead` object, the `index` attribute indicates the index of the bead in the CG topology, the `name` attribute gives the name of the bead, and the `aa_indices` and `aa_masses` attributes give the indices of atoms in the AA topology that map to the bead and the masses of the atomistic atoms.

In [9]:
for bead in tm.cg_beads:
    print(f"Bead {bead.index} : {bead.name}")
    print(f"    AA indices : {bead.aa_indices}")
    print(f"    AA masses  : {bead.aa_masses}")
    print(" ")

Bead 0 : Bpba
    AA indices : [ 0  9 10  1 11  2  3  4]
    AA masses  : [12.01078   1.007947  1.007947 12.01078   1.007947 12.01078  15.99943
 15.99943 ]
 
Bead 1 : C4
    AA indices : [ 5 12 13  6 14 15  7 16 17  8 18 19 20]
    AA masses  : [12.01078   1.007947  1.007947 12.01078   1.007947  1.007947 12.01078
  1.007947  1.007947 12.01078   1.007947  1.007947  1.007947]
 
Bead 2 : Bpba
    AA indices : [21 30 31 22 32 23 24 25]
    AA masses  : [12.01078   1.007947  1.007947 12.01078   1.007947 12.01078  15.99943
 15.99943 ]
 
Bead 3 : C4
    AA indices : [26 33 34 27 35 36 28 37 38 29 39 40 41]
    AA masses  : [12.01078   1.007947  1.007947 12.01078   1.007947  1.007947 12.01078
  1.007947  1.007947 12.01078   1.007947  1.007947  1.007947]
 
Bead 4 : Bpba
    AA indices : [42 51 52 43 53 44 45 46]
    AA masses  : [12.01078   1.007947  1.007947 12.01078   1.007947 12.01078  15.99943
 15.99943 ]
 
Bead 5 : C4
    AA indices : [47 54 55 48 56 57 49 58 59 50 60 61 62]
    AA masses 

When iterating through CG bonds, each item is a tuple of `CGBead` objects.

In [10]:
for bond in tm.cg_bonds:
    print(bond[0].index, bond[1].index)

0 6
0 2
0 1
2 4
2 3
4 8
4 5
6 7
8 9
10 16
10 12
10 11
12 14
12 13
14 18
14 15
16 17
18 19
20 21
21 22
23 24
24 25


If the `sim` module, developed by the Shell group at UCSB, is available, then the `TopologyMapper` object can be converted to objects used by `sim`.

In [26]:
# try importing sim
try:
    import sim
    print("sim module found")
    print(" ")
except ModuleNotFoundError:
    sim = None
    print("sim module not found")
    
if sim is not None:
    # convert to sim
    sim_topology = tm.to_sim()
    AtomNames = sim_topology.AtomNames
    AtomTypes = sim_topology.AtomTypes
    MolTypes = sim_topology.MolTypes
    PosMap = sim_topology.PosMap
    World = sim_topology.World
    System = sim_topology.System
    
    print(AtomNames)
    print(" ")
    print(AtomTypes)
    print(" ")
    print(MolTypes)

sim module found
 
['Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'Bpba', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'H2O', 'H2O']
 
{'Bpba': Bpba, 'C4': C4, 'H2O': H2O}
 
[0:[Bpba0, C41, Bpba2, C43, Bpba4, C45, Bpba6, C47, Bpba8, C49], 1:[C40, C41, C42], 2:[H2O0]]
